In [62]:
import anndata as ad
import os
import numpy as np
import torch
import pandas as pd
import scanpy as sc
import random
from scipy.sparse import issparse
from sklearn.decomposition import TruncatedSVD
import math

In [72]:
#######################################
# Configuration
#######################################

# Option to save results to CSV or just print them out
save_results = True 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

bench_data_dir = '/home/s2022310812/bench_data' 
dropout_data_dir = '/home/s2022310812/dropout_data'
datasets = ["IDC"]
model = "SPCS"
percents = ['30%']
samples = ['5']
trial_range = range(1, 6)  # 5 trials

Using device: cuda:0


In [73]:
#######################################
# SPCS Functions
#######################################

def calcRandPCA(tmat, d=10, seed=42):
    print("calcRandPCA: Starting PCA computation...")  # ADDED PRINT STATEMENT
    mean_vals = torch.mean(tmat, dim=0, keepdim=True)
    centered = tmat - mean_vals
    svd = TruncatedSVD(n_components=d, random_state=seed)
    centered_np = centered.cpu().numpy()
    X_pca = svd.fit_transform(centered_np)
    X_pca = torch.tensor(X_pca, dtype=torch.float32, device=device)
    print("calcRandPCA: PCA computation finished.")  # ADDED PRINT STATEMENT
    return X_pca

def calcPDist(X):
    print("calcPDist: Computing Pearson distance...")  # ADDED PRINT STATEMENT
    X_mean = torch.mean(X, dim=0)
    X_std = torch.std(X, dim=0)
    X_norm = (X - X_mean) / (X_std + 1e-9)
    corr = (X_norm @ X_norm.T) / X.shape[1]
    dist = 1 - corr
    print("calcPDist: Finished computing distance.")  # ADDED PRINT STATEMENT
    return dist

def calcPContribution(distMat):
    print("calcPContribution: Calculating pattern contribution...")  # ADDED PRINT STATEMENT
    rmat = torch.exp(-4.5 * (distMat ** 2))
    rmat[distMat > 1] = 0
    print("calcPContribution: Finished pattern contribution calculation.")  # ADDED PRINT STATEMENT
    return rmat

def findPNeighbors(mat_pcontribution, positions, tau_p=16):
    print("findPNeighbors: Finding pattern neighbors...")  # ADDED PRINT STATEMENT
    bars = positions['barcodes'].values
    results = {}
    for i, b in enumerate(bars):
        col = mat_pcontribution[:, i]
        nz = torch.nonzero(col > 0, as_tuple=True)[0]
        if len(nz) > 0:
            vals = col[nz]
            sorted_vals, sorted_idx = torch.sort(vals, descending=True)
            if tau_p > 0 and len(nz) > tau_p:
                chosen_idx = sorted_idx[:tau_p]
            else:
                chosen_idx = sorted_idx
            chosen_bars = bars[nz[chosen_idx].cpu().numpy()]
            chosen_vals = sorted_vals[:len(chosen_idx)].cpu().numpy()
            sum_contrib = chosen_vals.sum() + 1e-8
            ps_contrib = chosen_vals / sum_contrib
            df = pd.DataFrame({'barcodes': chosen_bars,
                               'p.contribution': chosen_vals,
                               'ps.contribution': ps_contrib})
            results[b] = df
        else:
            results[b] = None
    print("findPNeighbors: Finished finding pattern neighbors.")  # ADDED PRINT STATEMENT        
    return results

def GetPointNeighbors(px, py, order=2, is_keepself=False):
    x_range = np.arange(px - order, px + order + 1)
    x_range = x_range[x_range >= 0]
    y_range = np.arange(py - order, py + order + 1)
    y_range = y_range[y_range >= 0]
    points = []
    for xx in x_range:
        for yy in y_range:
            dist = abs(xx - px) + abs(yy - py)
            if dist <= order and (is_keepself or dist > 0):
                points.append((xx, yy, dist))
    points_df = pd.DataFrame(points, columns=['x', 'y', 'distance'])
    return points_df

def findSNeighbors(positions, is_hexa=False, tau_s=2):
    print("findSNeighbors: Finding spatial neighbors...")  # ADDED PRINT STATEMENT
    bars = positions['barcodes'].values
    if is_hexa:
        tau_s = tau_s * 2
    results = {}
    for i, b in enumerate(bars):
        px, py = positions.iloc[i]['st.x'], positions.iloc[i]['st.y']
        neighbors_candidates = GetPointNeighbors(px, py, order=tau_s, is_keepself=False)
        merged = neighbors_candidates.merge(positions, left_on=['x','y'], right_on=['st.x','st.y'], how='inner')
        if len(merged) > 0:
            s_contrib = 1.0 / (merged['distance'].values + 1e-9)
            s_sum = s_contrib.sum() + 1e-9
            ss_contrib = s_contrib / s_sum
            df = merged.copy()
            df['s.contribution'] = s_contrib
            df['ss.contribution'] = ss_contrib
            results[b] = df
        else:
            results[b] = None
    print("findSNeighbors: Finished finding spatial neighbors.")  # ADDED PRINT STATEMENT        
    return results

def getCSmoothedExp(exp, s_neighbors, p_neighbors, alpha, beta):
    print("getCSmoothedExp: Smoothing expression...")  # ADDED PRINT STATEMENT
    bars = list(p_neighbors.keys())
    ind_bars = {b: i for i, b in enumerate(bars)}
    G = exp.shape[0]
    N = exp.shape[1]

    exp_gpu = exp
    s_exp = torch.zeros((G, N), dtype=torch.float32, device=device)
    p_exp = torch.zeros((G, N), dtype=torch.float32, device=device)

    for i, b in enumerate(bars):
        if s_neighbors[b] is not None:
            nbars = s_neighbors[b]['barcodes'].values
            weights = s_neighbors[b]['ss.contribution'].values
            nb_idx = [ind_bars[x] for x in nbars]
            nb_idx_t = torch.tensor(nb_idx, device=device)
            w_t = torch.tensor(weights, device=device).unsqueeze(0)
            s_exp[:, i] = (exp_gpu[:, nb_idx_t] * w_t).sum(dim=1)
        if p_neighbors[b] is not None:
            nbars = p_neighbors[b]['barcodes'].values
            weights = p_neighbors[b]['ps.contribution'].values
            nb_idx = [ind_bars[x] for x in nbars]
            nb_idx_t = torch.tensor(nb_idx, device=device)
            w_t = torch.tensor(weights, device=device).unsqueeze(0)
            p_exp[:, i] = (exp_gpu[:, nb_idx_t] * w_t).sum(dim=1)

    exp_smoothed = exp_gpu * (1 - alpha) + (s_exp * beta + p_exp * (1 - beta)) * alpha
    print("getCSmoothedExp: Finished smoothing expression.")  # ADDED PRINT STATEMENT
    return exp_smoothed

def fillBlanks(exp, coord, is_hexa=False, tau_s=2, filling_threshold=0.5):

    print("fillBlanks: Starting fillBlanks...")  # ADDED PRINT STATEMENT
    ###############Change################
    # Convert coordinates in 'coord' to integers
    coord['st.x'] = coord['st.x'].round().astype(int)  # Ensure integer values
    coord['st.y'] = coord['st.y'].round().astype(int)

    # Now determine the ranges based on the integer coordinates
    x_min = coord['st.x'].min()
    x_max = coord['st.x'].max()
    y_min = coord['st.y'].min()
    y_max = coord['st.y'].max()
    ###############Change###############

    print(f"fillBlanks: Grid bounds x:[{x_min},{x_max}], y:[{y_min},{y_max}]")  # ADDED PRINT STATEMENT
    
    grid_points = []
    for xx in range(x_min, x_max+1):
        for yy in range(y_min, y_max+1):
            grid_points.append((xx, yy))
    grid_df = pd.DataFrame(grid_points, columns=['st.x', 'st.y'])

    print("fillBlanks: Created grid_df, now merging with coord...")  # ADDED PRINT STATEMENT

    merged = pd.merge(grid_df, coord, on=['st.x','st.y'], how='left', indicator=True)
    missing = merged[merged['_merge']=='left_only'].copy()
    missing['barcodes'] = ['SUPP'+str(i) for i in range(1, len(missing)+1)]

    print(f"fillBlanks: Found {len(missing)} missing spots.")  # ADDED PRINT STATEMENT

    if is_hexa:
        tau_s = tau_s * 2

    bars = coord['barcodes'].values
    ind_bars = {b: i for i, b in enumerate(bars)}

    s_exp = []
    s_cols = []
    for i, row in missing.iterrows():
        px, py = row['st.x'], row['st.y']
        neighbors_candidates = GetPointNeighbors(px, py, order=tau_s, is_keepself=False)
        if is_hexa:
            neighbors_candidates = neighbors_candidates[neighbors_candidates['distance']%2==0]

        ###############Change################
        # If needed, ensure neighbors_candidates are also ints:
        neighbors_candidates['x'] = neighbors_candidates['x'].astype(int)
        neighbors_candidates['y'] = neighbors_candidates['y'].astype(int)
        ###############Change################
        
        merged_nb = pd.merge(neighbors_candidates, coord, left_on=['x','y'], right_on=['st.x','st.y'], how='inner')
        if len(merged_nb) > 0:
            possible_count = len(neighbors_candidates)
            if len(merged_nb) >= filling_threshold * possible_count:
                s_contrib = 1.0 / (merged_nb['distance'].values + 1e-9)
                s_sum = s_contrib.sum() + 1e-9
                ss_contrib = s_contrib / s_sum
                nbars = merged_nb['barcodes'].values
                nb_idx = [ind_bars[x] for x in nbars]
                nb_idx_t = torch.tensor(nb_idx, device=device)
                w_t = torch.tensor(ss_contrib, device=device).unsqueeze(0)
                col_exp = (exp[:, nb_idx_t] * w_t).sum(dim=1, keepdim=True)
                s_exp.append(col_exp)
                s_cols.append(row['barcodes'])

    if len(s_exp) > 0:
        s_exp_matrix = torch.cat(s_exp, dim=1)
    else:
        s_exp_matrix = None

    print("fillBlanks: Completed fillBlanks.")  # ADDED PRINT STATEMENT    

    if s_exp_matrix is not None:
        colData = missing[missing['barcodes'].isin(s_cols)].copy()
    else:
        colData = pd.DataFrame(columns=['barcodes','st.x','st.y'])

    return s_exp_matrix, colData

def SPCS(data_tbl, coor_tbl, is_hexa=False, is_padding=False, tau_p=16, tau_s=2, alpha=0.6, beta=0.4, filling_thres=0.5, seed=42):

    print("SPCS: Starting SPCS computation...")  # ADDED PRINT STATEMENT
    
    data_mat = torch.tensor(data_tbl, dtype=torch.float32, device=device)
    rpca_res = calcRandPCA(data_mat.T, d=10, seed=seed)
    pdist_mat = calcPDist(rpca_res)
    rmat = calcPContribution(pdist_mat)

    positions = coor_tbl.copy()
    positions['barcodes'] = positions.index

    p_neighbors = findPNeighbors(rmat, positions, tau_p=tau_p)
    s_neighbors = findSNeighbors(positions, is_hexa, tau_s)

    exp_smoothed = getCSmoothedExp(data_mat, s_neighbors, p_neighbors, alpha, beta)

    if is_padding:
        print("SPCS: Padding is enabled, calling fillBlanks...")  # ADDED PRINT STATEMENT
        
        filled_exp, filled_colData = fillBlanks(exp_smoothed, positions, is_hexa, tau_s, filling_thres)
        if filled_exp is not None:
            combined = torch.cat([exp_smoothed, filled_exp], dim=1)
            combined_np = combined.cpu().numpy()
            print("SPCS: SPCS computation finished with padding.")
            return pd.DataFrame(combined_np, index=range(combined_np.shape[0]))
        else:
            print("SPCS: No blanks filled, SPCS computation finished.")
            return pd.DataFrame(exp_smoothed.cpu().numpy())
    else:
        print("SPCS: Padding not enabled, SPCS computation finished.")
        return pd.DataFrame(exp_smoothed.cpu().numpy())

In [74]:
###########################################
# Benchmarking Code
###########################################

for trial in trial_range:
    seed = trial
    print(f"\n===== Starting Trial {trial} with seed {seed} =====")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    for dataset in datasets:
        for percent in percents:
            for sample in samples:
                man_sc = {}
                bench_path = os.path.join(bench_data_dir,dataset,'adata')
                dropout_path = os.path.join(dropout_data_dir,dataset,percent+"-"+sample)

                if not os.path.exists(bench_path):
                    print(f"Warning: Benchmark path '{bench_path}' does not exist. Skipping {dataset}/{percent}-{sample}.")
                    continue
                if not os.path.exists(dropout_path):
                    print(f"Warning: Dropout path '{dropout_path}' does not exist. Skipping {dataset}/{percent}-{sample}.")
                    continue

                adata_names = [f for f in os.listdir(bench_path) if os.path.isfile(os.path.join(bench_path, f))]

                for adata_name in adata_names:
                    # Load and preprocess data
                    adata = ad.read_h5ad(os.path.join(bench_path,adata_name))
                    sc.pp.normalize_total(adata, target_sum=1e6, exclude_highly_expressed= False)
                    sc.pp.log1p(adata)
                    sc.pp.scale(adata)

                    adata_drop = ad.read_h5ad(os.path.join(dropout_path,adata_name))
                    sc.pp.normalize_total(adata_drop, target_sum=1e6, exclude_highly_expressed= False)
                    sc.pp.log1p(adata_drop)
                    sc.pp.scale(adata_drop)

                    if 'dropped' not in adata_drop.layers:
                        print(f"Warning: 'dropped' layer not found in {adata_name}, skipping.")
                        man_sc[dataset + "/" + adata_name] = np.nan
                        continue

                    dropped_mask = torch.tensor(adata_drop.layers['dropped'], dtype=torch.bool, device=device)

                    if issparse(adata_drop.X):
                        dense_data = adata_drop.X.toarray()
                    else:
                        dense_data = adata_drop.X

                    coor_tbl = pd.DataFrame(adata_drop.obsm['spatial'], index=adata_drop.obs.index, columns=['st.x','st.y'])
                    data_tbl = dense_data.T  # genes x spots

                    # Run SPCS with current trial's seed
                    print(f"Running SPCS on {adata_name}...")  # ADDED PRINT STATEMENT
                    smoothed_data_df = SPCS(data_tbl, coor_tbl, is_hexa=False, is_padding=False, tau_p=40, tau_s=4, alpha=0.6, beta=0.4, filling_thres=0.5, seed=seed)
                    adata2 = smoothed_data_df.to_numpy().T.astype(np.float32)

                    if issparse(adata.X):
                        adata_X = adata.X.toarray().astype(np.float32)
                    else:
                        adata_X = adata.X.astype(np.float32)
                    adata_tensor = torch.tensor(adata_X, dtype=torch.float32, device=device)

                    adata2_tensor = torch.tensor(adata2, dtype=torch.float32, device=device)

                    dropped_indices = dropped_mask.nonzero(as_tuple=True)

                    adata_values = adata_tensor[dropped_indices]
                    adata2_values = adata2_tensor[dropped_indices]

                    adata_values_cpu = adata_values.cpu().numpy()
                    adata2_values_cpu = adata2_values.cpu().numpy()

                    manhattan_distance = np.abs(adata_values_cpu - adata2_values_cpu).sum()
                    num_values = adata_values_cpu.size
                    manhattan_score = manhattan_distance / num_values if num_values > 0 else np.nan

                    man_sc[dataset + "/" + adata_name] = manhattan_score
                    print(f"Processed {adata_name}, Manhattan score: {manhattan_score}")

                # After processing all adata_names
                if not man_sc:
                    print(f"No results for {dataset}/{percent}-{sample} in Trial {trial}, skipping CSV write.")
                    continue

                row_name = f"Trial : {trial}"
                my_dir = os.path.join('/home/s2022310812/bench_results', model, dataset)
                csv_path = os.path.join(my_dir, percent+"-"+sample +"_result_Ft40s4_final.csv")
                os.makedirs(my_dir, exist_ok=True)

                columns = [key.split("/")[-1].split(".")[0] for key in man_sc.keys()]
                values = list(man_sc.values())
                data = pd.DataFrame([values], columns=columns, index=[row_name])

                # Only save if save_results = True
                if save_results:
                    # Attempt to read existing data
                    existing_data = None
                    if os.path.exists(csv_path):
                        print(f"CSV file '{csv_path}' exists. Attempting to read it.")
                        try:
                            existing_data = pd.read_csv(csv_path, index_col=0)
                            print(f"Successfully read existing CSV with shape {existing_data.shape}.")
                        except Exception as e:
                            print(f"Warning: Error reading existing CSV '{csv_path}': {e}")
                            print("A new CSV will be created.")
                            existing_data = None
                    else:
                        print(f"CSV file '{csv_path}' does not exist. A new one will be created.")

                    if existing_data is not None:
                        updated_data = pd.concat([existing_data, data])
                    else:
                        updated_data = data

                    # Try saving the CSV
                    print(f"Attempting to save results to '{csv_path}'...")
                    try:
                        updated_data.to_csv(csv_path)
                        print(f"Saved results to '{csv_path}'")
                    except Exception as e:
                        print(f"Error saving CSV '{csv_path}': {e}")
                else:
                    print("save_results=False, not saving results to CSV. Results for this trial:")
                    print(data)

    print(f"===== Completed Trial {trial} =====\n")

print("Benchmarking completed.")


===== Starting Trial 1 with seed 1 =====
Running SPCS on NCBI785.h5ad...
SPCS: Starting SPCS computation...
calcRandPCA: Starting PCA computation...


/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


calcRandPCA: PCA computation finished.
calcPDist: Computing Pearson distance...
calcPDist: Finished computing distance.
calcPContribution: Calculating pattern contribution...
calcPContribution: Finished pattern contribution calculation.
findPNeighbors: Finding pattern neighbors...
findPNeighbors: Finished finding pattern neighbors.
findSNeighbors: Finding spatial neighbors...
findSNeighbors: Finished finding spatial neighbors.
getCSmoothedExp: Smoothing expression...
getCSmoothedExp: Finished smoothing expression.
SPCS: Padding not enabled, SPCS computation finished.
Processed NCBI785.h5ad, Manhattan score: 1.3616162441307884
Running SPCS on NCBI783.h5ad...
SPCS: Starting SPCS computation...
calcRandPCA: Starting PCA computation...


/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


calcRandPCA: PCA computation finished.
calcPDist: Computing Pearson distance...
calcPDist: Finished computing distance.
calcPContribution: Calculating pattern contribution...
calcPContribution: Finished pattern contribution calculation.
findPNeighbors: Finding pattern neighbors...
findPNeighbors: Finished finding pattern neighbors.
findSNeighbors: Finding spatial neighbors...
findSNeighbors: Finished finding spatial neighbors.
getCSmoothedExp: Smoothing expression...
getCSmoothedExp: Finished smoothing expression.
SPCS: Padding not enabled, SPCS computation finished.
Processed NCBI783.h5ad, Manhattan score: 1.5264226170776105


...

calcRandPCA: PCA computation finished.
calcPDist: Computing Pearson distance...
calcPDist: Finished computing distance.
calcPContribution: Calculating pattern contribution...
calcPContribution: Finished pattern contribution calculation.
findPNeighbors: Finding pattern neighbors...
findPNeighbors: Finished finding pattern neighbors.
findSNeighbors: Finding spatial neighbors...
findSNeighbors: Finished finding spatial neighbors.
getCSmoothedExp: Smoothing expression...
getCSmoothedExp: Finished smoothing expression.
SPCS: Padding not enabled, SPCS computation finished.
Processed NCBI783.h5ad, Manhattan score: 1.5241112103454653


/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


Running SPCS on TENX95.h5ad...
SPCS: Starting SPCS computation...
calcRandPCA: Starting PCA computation...
calcRandPCA: PCA computation finished.
calcPDist: Computing Pearson distance...
calcPDist: Finished computing distance.
calcPContribution: Calculating pattern contribution...
calcPContribution: Finished pattern contribution calculation.
findPNeighbors: Finding pattern neighbors...
findPNeighbors: Finished finding pattern neighbors.
findSNeighbors: Finding spatial neighbors...
findSNeighbors: Finished finding spatial neighbors.
getCSmoothedExp: Smoothing expression...
getCSmoothedExp: Finished smoothing expression.
SPCS: Padding not enabled, SPCS computation finished.
Processed TENX95.h5ad, Manhattan score: 1.5941966926377895


/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/s2022310812/.conda/envs/SPCS_v2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


Running SPCS on TENX99.h5ad...
SPCS: Starting SPCS computation...
calcRandPCA: Starting PCA computation...
calcRandPCA: PCA computation finished.
calcPDist: Computing Pearson distance...
calcPDist: Finished computing distance.
calcPContribution: Calculating pattern contribution...
calcPContribution: Finished pattern contribution calculation.
findPNeighbors: Finding pattern neighbors...
findPNeighbors: Finished finding pattern neighbors.
findSNeighbors: Finding spatial neighbors...
findSNeighbors: Finished finding spatial neighbors.
getCSmoothedExp: Smoothing expression...
getCSmoothedExp: Finished smoothing expression.
SPCS: Padding not enabled, SPCS computation finished.
Processed TENX99.h5ad, Manhattan score: 1.6218502065103826
CSV file '/home/s2022310812/bench_results/SPCS/IDC/30%-5_result_Ft40s4_final.csv' exists. Attempting to read it.
Successfully read existing CSV with shape (4, 4).
Attempting to save results to '/home/s2022310812/bench_results/SPCS/IDC/30%-5_result_Ft40s4_final